In [1]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from scipy.interpolate import UnivariateSpline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import math
from fbprophet import Prophet
import matplotlib.pyplot as plt

Importing plotly failed. Interactive plots will not work.


In [2]:
def getStates():
    ds = pd.read_csv('data/test.csv')
    states = ds['Province_State'][:50].values   
    return states

In [3]:
def distance(x, y, side_check=False):
    if side_check:
        if y > x:
            return 0
    b = y + x
    i = b / 2
    return math.sqrt((x - i)**2 + (y - i)**2)

In [4]:
def all_distance(data, side_check=False):
    rtn = []
    for i in range(len(data)):
        rtn.append(distance(i, data[i], side_check=side_check))
    return rtn

In [5]:
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

In [6]:
degree = 3
states = getStates()
start = 213
alpha = 5.2
window = 25
states2idx = {}
for i in range(len(states)):
    states2idx[states[i]] = i

In [7]:
feature = 'Deaths'
res = []
total = 0

for i in range(len(states)):
    state = states[i]
    ds = pd.read_csv('data/train_round2.csv')
    ds = ds[ds['Province_State'] == state]
    raw = ds[feature].values
    value = savgol_filter(raw.reshape(-1), window, degree) #smooth data
    
###===================================================================###   
### no saturation no negative slope ###
    scale = raw / raw[-1] * 225
    distances = all_distance(scale)
    diff = sum(distances[-30:])
    max_point = distances.index(max(distances))       
    
    x = [[i] for i in range(len(value))]
    
    y = np.array([value[start:]]).reshape(-1, 1)
    x = [[i + start] for i in range(225-start)]
    model = Ridge(alpha=alpha)
    model.fit(x, y)
    x_test = [[i + 225] for i in range(21)]
    y_hat = model.predict(x_test)
 
    slope = 21 / (y_hat[-1] - y_hat[0])
    smooth_slope = 1 / (value[-1] - value[-2])

    diff = abs(smooth_slope - slope)
    if diff > 0.5:
        y = np.array([value[start - 28:]]).reshape(-1, 1)
        x = [[i + start - 28] for i in range(225 - start + 28)]
        model = Ridge(alpha=alpha)
        model.fit(x, y)
        y_hat = model.predict(x_test)

        sub = 0.1 * smooth_slope + 0.9 * slope
        cur_slope = 21 / (y_hat[-1] - y_hat[0])
        y_hat = (y_hat-y_hat[0]) / cur_slope * sub + y_hat[0]

    res.append(y_hat)

In [8]:
rerange = []
for i in range(len(res[0])):
    for j in range(len(res)):
        rerange.append(res[j][i])
#print(rerange)
df = pd.DataFrame(rerange)
df.to_csv("{}.csv".format(feature),index=False,sep=',')